# Pull data from Hybrid-Analysis using their API and make a model to predict whether traffic is malicious or no

### conda install requests

In [1]:
import requests
import json

### Set headers for the API request

In [2]:
parameters = {
    "api-key": "o1txu2cfedff17a6smm7u4ifd1644c93ifhj46b824096cd5xh7id5qse2613f06",
    "accept-encoding" : "gzip",
    "user-agent" : "Falcon Sandbox"
}

### Pull from feed

In [3]:
response = requests.get("https://www.hybrid-analysis.com/feed?json", headers=parameters)
## log out the return code and any headers - not required for this problem but useful
print("response code: ", response.status_code, "content length is ", len(response.content))
print("headers: ", json.dumps(response.headers.__dict__, sort_keys=True, indent=4))

response code:  200 content length is  3589769
headers:  {
    "_store": {
        "access-control-allow-origin": [
            "Access-Control-Allow-Origin",
            "*"
        ],
        "cache-control": [
            "Cache-Control",
            "max-age=0, private, must-revalidate, max-age=2592000"
        ],
        "cf-cache-status": [
            "CF-Cache-Status",
            "DYNAMIC"
        ],
        "cf-ray": [
            "CF-RAY",
            "63c01b010fdac7dd-DEN"
        ],
        "cf-request-id": [
            "cf-request-id",
            "094bef34a40000c7ddfa8d1000000001"
        ],
        "connection": [
            "Connection",
            "keep-alive"
        ],
        "content-encoding": [
            "Content-Encoding",
            "gzip"
        ],
        "content-security-policy": [
            "Content-Security-Policy",
            "default-src 'none'; connect-src 'self' *.twitter.com; script-src 'self' *.google.com *.gstatic.com *.google-analytics.

In [4]:
json_response = response.json()
datasets = json_response['data']
print(datasets)

[{'md5': '50de299bc20e4b1f5a82e2d06f5a34ec', 'sha1': '9ae6a25aa146ebdbecf0e53f91ad30fff348275b', 'sha256': '561bf8a01099667d5e47c63acf0a30d02627004459da7a34ff12a9ca7e9fda65', 'isinteresting': False, 'analysis_start_time': '2021-04-07 03:09:13', 'threatscore': 20, 'threatlevel': 0, 'threatlevel_human': 'no specific threat', 'avdetect': 0, 'isunknown': False, 'submitname': 'choku_nuki_umi.exe', 'isurlanalysis': False, 'size': 617984, 'type': 'PE32 executable (GUI) Intel 80386, for MS Windows', 'environmentId': '120', 'environmentDescription': 'Windows 7 64 bit', 'sharedanalysis': False, 'isreliable': True, 'reporturl': '/sample/561bf8a01099667d5e47c63acf0a30d02627004459da7a34ff12a9ca7e9fda65/606d2257a2bc4316e72d3eba', 'vt_detect': 0, 'ms_detect': 0, 'process_list': [{'uid': '00064869-00003404', 'name': 'choku_nuki_umi.exe', 'normalizedpath': 'C:\\choku_nuki_umi.exe', 'commandline': '', 'sha256': '561bf8a01099667d5e47c63acf0a30d02627004459da7a34ff12a9ca7e9fda65', 'av_matched': 0, 'av_tota

In [5]:
json_response = response.json()
datasets = json_response['data']
print (len(datasets))
print ("first one is of type ", type(datasets[0]))
#print ("first one is ", datasets[0])
print (datasets[0].keys())
print()
#print ("first one is ", json.dumps(datasets[0], sort_keys=True, indent=4))
print ("is interesting: ", datasets[0].get("isinteresting"), " threatscore:", datasets[0].get("threatscore"), "reporturl: ", datasets[0].get("reporturl"), " sha256:", datasets[0].get("sha256"))

200
first one is of type  <class 'dict'>
dict_keys(['md5', 'sha1', 'sha256', 'isinteresting', 'analysis_start_time', 'threatscore', 'threatlevel', 'threatlevel_human', 'avdetect', 'isunknown', 'submitname', 'isurlanalysis', 'size', 'type', 'environmentId', 'environmentDescription', 'sharedanalysis', 'isreliable', 'reporturl', 'vt_detect', 'ms_detect', 'process_list', 'extracted_files', 'ssdeep'])

is interesting:  False  threatscore: 20 reporturl:  /sample/561bf8a01099667d5e47c63acf0a30d02627004459da7a34ff12a9ca7e9fda65/606d2257a2bc4316e72d3eba  sha256: 561bf8a01099667d5e47c63acf0a30d02627004459da7a34ff12a9ca7e9fda65


### This for loop is just practice for a similar for loop later on. Not used in model

In [6]:
for aThing in datasets:
    print ("is interesting: ", aThing.get("isinteresting"), " threatscore:", aThing.get("threatscore"),  " reporturl:", aThing.get("reporturl"),  " sha256:", aThing.get("sha256"))

is interesting:  False  threatscore: 20  reporturl: /sample/561bf8a01099667d5e47c63acf0a30d02627004459da7a34ff12a9ca7e9fda65/606d2257a2bc4316e72d3eba  sha256: 561bf8a01099667d5e47c63acf0a30d02627004459da7a34ff12a9ca7e9fda65
is interesting:  False  threatscore: 0  reporturl: /sample/a6c4821f7fc6c1f29360dc270a628eb7d98b4b538ece064fce8456cf01fe84b3/606d2213c46a521ed93230b6  sha256: a6c4821f7fc6c1f29360dc270a628eb7d98b4b538ece064fce8456cf01fe84b3
is interesting:  True  threatscore: 85  reporturl: /sample/970da3b5006884e96e8dbc91af2d597f5502b2876974e1454815068bc7eec791/606d2212ff579a39ac17389d  sha256: 970da3b5006884e96e8dbc91af2d597f5502b2876974e1454815068bc7eec791
is interesting:  True  threatscore: 85  reporturl: /sample/a472ed9ce9423cb9d22d016b41a8926a03cb01d283ad0cc15841ceffeac37b3d/606d2212dd1daa5a5b6ef9a7  sha256: a472ed9ce9423cb9d22d016b41a8926a03cb01d283ad0cc15841ceffeac37b3d
is interesting:  False  threatscore: 100  reporturl: /sample/79ef2caff69b87e457c5bb29652cb97fc251f0233c3f3f

### Format the data into a dataframe

In [7]:
import pandas as pd
df = pd.DataFrame(datasets)
df

,md5,sha1,sha256,isinteresting,analysis_start_time,threatscore,threatlevel,threatlevel_human,avdetect,isunknown,...,public_references,domains,hosts,hosts_geo,compromised_hosts,domains_capped,hosts_capped,et_alerts_total,et_alerts_real_total,et_alerts
0,50de299bc20e4b1f5a82e2d06f5a34ec,9ae6a25aa146ebdbecf0e53f91ad30fff348275b,561bf8a01099667d5e47c63acf0a30d02627004459da7a...,False,2021-04-07 03:09:13,20,0,no specific threat,0.0,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ec6991f6ad5febe38319205f5b0c69ce,8f24fcc9fe5bdf200e2471658e9935563269e444,a6c4821f7fc6c1f29360dc270a628eb7d98b4b538ece06...,False,2021-04-07 03:08:05,0,0,no specific threat,NaN,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a64b513444746ce39864faf931ffc00e,d7ab9db59e50a8e970b744d84fe7f000f087a05b,970da3b5006884e96e8dbc91af2d597f5502b2876974e1...,True,2021-04-07 03:08:05,85,2,malicious,NaN,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ab7fa2166271146e7ee06ea36a213ec4,88b362aa109329fd5349dfda436b1191b4458243,a472ed9ce9423cb9d22d016b41a8926a03cb01d283ad0c...,True,2021-04-07 03:08:05,85,2,malicious,NaN,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0ddee377b166a4a00fe1fe24ccdccc80,e61197f0ca2bd1ea753a98c017f4f8a4326bd2fe,79ef2caff69b87e457c5bb29652cb97fc251f0233c3f3f...,False,2021-04-07 03:07:23,100,2,malicious,51.0,False,...,[https://twitter.com/BushidoToken/status/12526...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,f02c31d9054c7a0619d9ad7e0a2c269f,e1211a1856101f8125631a62f29a54d86f072e31,cb01e4772b2177b11485e4189faa7f81f5c90c01f31d3d...,False,2021-04-07 01:27:27,35,1,suspicious,NaN,True,...,NaN,"[auspiciousalleged.info, away.vk.com, ocsp.pki...","[172.217.12.196, 172.217.10.3, 87.240.190.64, ...","[{'ip': '172.217.12.196', 'lat': '40.8584', 'l...","[87.240.190.64, 87.240.190.78, 93.186.225.208]",NaN,NaN,NaN,NaN,NaN
196,e362981170e65bacac78b2bd30c41311,2c41388a8011d544018a5b06c560fe49a0738aad,a7526e44addf50a57f7e855e56cca7b9641cc909ad4fb8...,False,2021-04-07 01:25:08,0,2,malicious,NaN,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,c7420b364b49c5aa7c761d2cd95efa39,94dcb34631bff73e8842e3415b2841265bc03007,a73781adb259d4f63b1e54ff4f8df285b82eeae11251db...,False,2021-04-07 01:25:01,70,2,malicious,0.0,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,15cb46f9cd8f620e9e6162ee7d66add0,d64e4c210cc6163d5a5adef49087c3b76a87b037,db0ebdbd91be3713b5fbf327fd35b399a9c84887c2c2b0...,True,2021-04-07 01:24:09,85,2,malicious,NaN,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Look at the reporturl column to see how we need to split it up so we can put the values in a for loop to pull each individual report

In [8]:
df["reporturl"]

0      /sample/561bf8a01099667d5e47c63acf0a30d0262700...
1      /sample/a6c4821f7fc6c1f29360dc270a628eb7d98b4b...
2      /sample/970da3b5006884e96e8dbc91af2d597f5502b2...
3      /sample/a472ed9ce9423cb9d22d016b41a8926a03cb01...
4      /sample/79ef2caff69b87e457c5bb29652cb97fc251f0...
                             ...                        
195    /sample/cb01e4772b2177b11485e4189faa7f81f5c90c...
196    /sample/a7526e44addf50a57f7e855e56cca7b9641cc9...
197    /sample/a73781adb259d4f63b1e54ff4f8df285b82eea...
198    /sample/db0ebdbd91be3713b5fbf327fd35b399a9c848...
199    /sample/e8784748119d79bf5d3f5c274497c168167652...
Name: reporturl, Length: 200, dtype: object

### Split the report url to seperate out jobid

In [9]:
#Filter out /sample/ from reporturl column
df["jobid"]=df["reporturl"].str.split("/", expand=True)[3]
list(df)

['md5',
 'sha1',
 'sha256',
 'isinteresting',
 'analysis_start_time',
 'threatscore',
 'threatlevel',
 'threatlevel_human',
 'avdetect',
 'isunknown',
 'submitname',
 'isurlanalysis',
 'size',
 'type',
 'environmentId',
 'environmentDescription',
 'sharedanalysis',
 'isreliable',
 'reporturl',
 'vt_detect',
 'ms_detect',
 'process_list',
 'extracted_files',
 'ssdeep',
 'tags',
 'vxfamily',
 'public_references',
 'domains',
 'hosts',
 'hosts_geo',
 'compromised_hosts',
 'domains_capped',
 'hosts_capped',
 'et_alerts_total',
 'et_alerts_real_total',
 'et_alerts',
 'jobid']

### Combine url with job id so we can insert into for loop to pull all the matching reports from the feed.

In [10]:
df["url"]= "https://www.hybrid-analysis.com/api/v2/report/"+df["jobid"]+"/summary"
df["url"]

0      https://www.hybrid-analysis.com/api/v2/report/...
1      https://www.hybrid-analysis.com/api/v2/report/...
2      https://www.hybrid-analysis.com/api/v2/report/...
3      https://www.hybrid-analysis.com/api/v2/report/...
4      https://www.hybrid-analysis.com/api/v2/report/...
                             ...                        
195    https://www.hybrid-analysis.com/api/v2/report/...
196    https://www.hybrid-analysis.com/api/v2/report/...
197    https://www.hybrid-analysis.com/api/v2/report/...
198    https://www.hybrid-analysis.com/api/v2/report/...
199    https://www.hybrid-analysis.com/api/v2/report/...
Name: url, Length: 200, dtype: object

### DO NOT USE THIS ONE: Use api to pull report summary data 

In [11]:
#for index, row in df.iterrows():
    ## log out the return code and any headers - not required for this problem but useful
    #print(row["url"])
    #response = requests.get(row["url"], headers=parameters)
    #print("response code: ", response.status_code, "content length is ", len(response.content))
    #print("headers: ", json.dumps(response.headers.__dict__, sort_keys=True, indent=4))
    #json_response = response.json()
    #report = json_response['data']
    #print(json_response)
    
    #df.at[index,"summary"] = json_response

### This call puts the json data and puts it into a new "summary" column

In [12]:
def get_summary(row):
    #print(row)
    response = requests.get(row["url"], headers=parameters)
    print("response code: ", response.status_code, "content length is ", len(response.content))
    #print("headers: ", json.dumps(response.headers.__dict__, sort_keys=True, indent=4))
    json_response = response.json()
    #report = json_response['data']
    #print(json_response)
    
    return json_response
    
df["summary"] = df.apply(get_summary, axis = 1)

response code:  200 content length is  3510
response code:  200 content length is  11575
response code:  200 content length is  6861
response code:  200 content length is  6891
response code:  200 content length is  5181
response code:  200 content length is  7237
response code:  200 content length is  31893
response code:  200 content length is  31772
response code:  200 content length is  14313
response code:  200 content length is  31576
response code:  200 content length is  31922
response code:  200 content length is  31597
response code:  200 content length is  30779
response code:  200 content length is  6861
response code:  200 content length is  6891
response code:  200 content length is  1949
response code:  200 content length is  4145
response code:  200 content length is  31500
response code:  200 content length is  21233
response code:  200 content length is  14689
response code:  200 content length is  20932
response code:  200 content length is  31678
response code:  200

### Make sure it worked and added the new summary column

In [13]:
df["summary"]

0      {'job_id': '606d2257a2bc4316e72d3eba', 'enviro...
1      {'job_id': '606d2213c46a521ed93230b6', 'enviro...
2      {'job_id': '606d2212ff579a39ac17389d', 'enviro...
3      {'job_id': '606d2212dd1daa5a5b6ef9a7', 'enviro...
4      {'job_id': '606d21e83d93b675940e44fd', 'enviro...
                             ...                        
195    {'job_id': '606d0a784ba30959525919d1', 'enviro...
196    {'job_id': '606d09f03bc2453d2e4c82c9', 'enviro...
197    {'job_id': '606d09ebe017526dcb31c7e4', 'enviro...
198    {'job_id': '606d09b32948f93bed09e3dd', 'enviro...
199    {'job_id': '606d09b2f7cb2561923a8084', 'enviro...
Name: summary, Length: 200, dtype: object

### Just checking the summary column type

In [14]:
print(type(df["summary"]))
print(df["summary"].dtypes)

<class 'pandas.core.series.Series'>
object


### This makes sure no failed reports are included. Aka if we run out of daily requests those aren't added into the dataframe. The block below shows the length of each summary.

In [15]:
df['sum_len'] = df['summary'].str.len()
print(df['sum_len'])

0      41
1      41
2      41
3      41
4      41
       ..
195    41
196    41
197    41
198    41
199    41
Name: sum_len, Length: 200, dtype: int64


### This block gets rid of all the summary's that didn't come through. their length is less than 2

In [16]:
df = df[df.sum_len > 2]
print(df['sum_len'])

0      41
1      41
2      41
3      41
4      41
       ..
195    41
196    41
197    41
198    41
199    41
Name: sum_len, Length: 200, dtype: int64


### Create function to make a new row counting the number of mitre attacks

In [17]:
#for index, row in df.iterrows():
    # log out the return code and any headers - not required for this problem but useful
    #print(len(row["summary"]["mitre_attcks"]))
    
def get_num_mitre_attcks(row):
    return len(row["summary"]["mitre_attcks"])

    
df["num_mitre_attcks"] = df.apply(get_num_mitre_attcks, axis = 1)
print(df['num_mitre_attcks'])

0       0
1       0
2      14
3      14
4       6
       ..
195     4
196     4
197     9
198    14
199    14
Name: num_mitre_attcks, Length: 200, dtype: int64


# Model Building
### Import needed packages

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm


%matplotlib inline
plt.style.use('ggplot')
import warnings; warnings.simplefilter('ignore')

# Make a copy of df. We can only access so many pulls in one day from the api so it is imperitive to work "offline" to keep testing

In [19]:
df2 = df.copy()

### Make the 'isinteresting' column a binary

In [20]:
df2 = df2.reset_index()
np.isnan(df2.any()) #and gets False
np.isfinite(df2.all()) #and gets True
df2

,index,md5,sha1,sha256,isinteresting,analysis_start_time,threatscore,threatlevel,threatlevel_human,avdetect,...,domains_capped,hosts_capped,et_alerts_total,et_alerts_real_total,et_alerts,jobid,url,summary,sum_len,num_mitre_attcks
0,0,50de299bc20e4b1f5a82e2d06f5a34ec,9ae6a25aa146ebdbecf0e53f91ad30fff348275b,561bf8a01099667d5e47c63acf0a30d02627004459da7a...,False,2021-04-07 03:09:13,20,0,no specific threat,0.0,...,NaN,NaN,NaN,NaN,NaN,606d2257a2bc4316e72d3eba,https://www.hybrid-analysis.com/api/v2/report/...,"{'job_id': '606d2257a2bc4316e72d3eba', 'enviro...",41,0
1,1,ec6991f6ad5febe38319205f5b0c69ce,8f24fcc9fe5bdf200e2471658e9935563269e444,a6c4821f7fc6c1f29360dc270a628eb7d98b4b538ece06...,False,2021-04-07 03:08:05,0,0,no specific threat,NaN,...,NaN,NaN,NaN,NaN,NaN,606d2213c46a521ed93230b6,https://www.hybrid-analysis.com/api/v2/report/...,"{'job_id': '606d2213c46a521ed93230b6', 'enviro...",41,0
2,2,a64b513444746ce39864faf931ffc00e,d7ab9db59e50a8e970b744d84fe7f000f087a05b,970da3b5006884e96e8dbc91af2d597f5502b2876974e1...,True,2021-04-07 03:08:05,85,2,malicious,NaN,...,NaN,NaN,NaN,NaN,NaN,606d2212ff579a39ac17389d,https://www.hybrid-analysis.com/api/v2/report/...,"{'job_id': '606d2212ff579a39ac17389d', 'enviro...",41,14
3,3,ab7fa2166271146e7ee06ea36a213ec4,88b362aa109329fd5349dfda436b1191b4458243,a472ed9ce9423cb9d22d016b41a8926a03cb01d283ad0c...,True,2021-04-07 03:08:05,85,2,malicious,NaN,...,NaN,NaN,NaN,NaN,NaN,606d2212dd1daa5a5b6ef9a7,https://www.hybrid-analysis.com/api/v2/report/...,"{'job_id': '606d2212dd1daa5a5b6ef9a7', 'enviro...",41,14
4,4,0ddee377b166a4a00fe1fe24ccdccc80,e61197f0ca2bd1ea753a98c017f4f8a4326bd2fe,79ef2caff69b87e457c5bb29652cb97fc251f0233c3f3f...,False,2021-04-07 03:07:23,100,2,malicious,51.0,...,NaN,NaN,NaN,NaN,NaN,606d21e83d93b675940e44fd,https://www.hybrid-analysis.com/api/v2/report/...,"{'job_id': '606d21e83d93b675940e44fd', 'enviro...",41,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,195,f02c31d9054c7a0619d9ad7e0a2c269f,e1211a1856101f8125631a62f29a54d86f072e31,cb01e4772b2177b11485e4189faa7f81f5c90c01f31d3d...,False,2021-04-07 01:27:27,35,1,suspicious,NaN,...,NaN,NaN,NaN,NaN,NaN,606d0a784ba30959525919d1,https://www.hybrid-analysis.com/api/v2/report/...,"{'job_id': '606d0a784ba30959525919d1', 'enviro...",41,4
196,196,e362981170e65bacac78b2bd30c41311,2c41388a8011d544018a5b06c560fe49a0738aad,a7526e44addf50a57f7e855e56cca7b9641cc909ad4fb8...,False,2021-04-07 01:25:08,0,2,malicious,NaN,...,NaN,NaN,NaN,NaN,NaN,606d09f03bc2453d2e4c82c9,https://www.hybrid-analysis.com/api/v2/report/...,"{'job_id': '606d09f03bc2453d2e4c82c9', 'enviro...",41,4
197,197,c7420b364b49c5aa7c761d2cd95efa39,94dcb34631bff73e8842e3415b2841265bc03007,a73781adb259d4f63b1e54ff4f8df285b82eeae11251db...,False,2021-04-07 01:25:01,70,2,malicious,0.0,...,NaN,NaN,NaN,NaN,NaN,606d09ebe017526dcb31c7e4,https://www.hybrid-analysis.com/api/v2/report/...,"{'job_id': '606d09ebe017526dcb31c7e4', 'enviro...",41,9
198,198,15cb46f9cd8f620e9e6162ee7d66add0,d64e4c210cc6163d5a5adef49087c3b76a87b037,db0ebdbd91be3713b5fbf327fd35b399a9c84887c2c2b0...,True,2021-04-07 01:24:09,85,2,malicious,NaN,...,NaN,NaN,NaN,NaN,NaN,606d09b32948f93bed09e3dd,https://www.hybrid-analysis.com/api/v2/report/...,"{'job_id': '606d09b32948f93bed09e3dd', 'enviro...",41,14


### Check how many interesting rows there are

In [21]:
df2.head()

,index,md5,sha1,sha256,isinteresting,analysis_start_time,threatscore,threatlevel,threatlevel_human,avdetect,...,domains_capped,hosts_capped,et_alerts_total,et_alerts_real_total,et_alerts,jobid,url,summary,sum_len,num_mitre_attcks
0,0,50de299bc20e4b1f5a82e2d06f5a34ec,9ae6a25aa146ebdbecf0e53f91ad30fff348275b,561bf8a01099667d5e47c63acf0a30d02627004459da7a...,False,2021-04-07 03:09:13,20,0,no specific threat,0.0,...,NaN,NaN,NaN,NaN,NaN,606d2257a2bc4316e72d3eba,https://www.hybrid-analysis.com/api/v2/report/...,"{'job_id': '606d2257a2bc4316e72d3eba', 'enviro...",41,0
1,1,ec6991f6ad5febe38319205f5b0c69ce,8f24fcc9fe5bdf200e2471658e9935563269e444,a6c4821f7fc6c1f29360dc270a628eb7d98b4b538ece06...,False,2021-04-07 03:08:05,0,0,no specific threat,NaN,...,NaN,NaN,NaN,NaN,NaN,606d2213c46a521ed93230b6,https://www.hybrid-analysis.com/api/v2/report/...,"{'job_id': '606d2213c46a521ed93230b6', 'enviro...",41,0
2,2,a64b513444746ce39864faf931ffc00e,d7ab9db59e50a8e970b744d84fe7f000f087a05b,970da3b5006884e96e8dbc91af2d597f5502b2876974e1...,True,2021-04-07 03:08:05,85,2,malicious,NaN,...,NaN,NaN,NaN,NaN,NaN,606d2212ff579a39ac17389d,https://www.hybrid-analysis.com/api/v2/report/...,"{'job_id': '606d2212ff579a39ac17389d', 'enviro...",41,14
3,3,ab7fa2166271146e7ee06ea36a213ec4,88b362aa109329fd5349dfda436b1191b4458243,a472ed9ce9423cb9d22d016b41a8926a03cb01d283ad0c...,True,2021-04-07 03:08:05,85,2,malicious,NaN,...,NaN,NaN,NaN,NaN,NaN,606d2212dd1daa5a5b6ef9a7,https://www.hybrid-analysis.com/api/v2/report/...,"{'job_id': '606d2212dd1daa5a5b6ef9a7', 'enviro...",41,14
4,4,0ddee377b166a4a00fe1fe24ccdccc80,e61197f0ca2bd1ea753a98c017f4f8a4326bd2fe,79ef2caff69b87e457c5bb29652cb97fc251f0233c3f3f...,False,2021-04-07 03:07:23,100,2,malicious,51.0,...,NaN,NaN,NaN,NaN,NaN,606d21e83d93b675940e44fd,https://www.hybrid-analysis.com/api/v2/report/...,"{'job_id': '606d21e83d93b675940e44fd', 'enviro...",41,6


In [22]:
isinteresting = df2['isinteresting'].astype(int)
sum(isinteresting)

36

### Try making nan values 0

In [23]:
def clean_dataset(df2):
    assert isinstance(df2, pd.DataFrame), "df needs to be a pd.DataFrame"
    df2.dropna(inplace=True)
    indices_to_keep = ~df2.isin([np.nan, np.inf, -np.inf]).any(1)
    return df2[indices_to_keep].astype(np.float64)

### Add input variables and split the testing data on a 70/30 split

In [24]:
X=df2[['threatscore', 'threatlevel', 'num_mitre_attcks']]  # Features
y=df2[['isinteresting']]  # Labels
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test
print(y.astype(int))

     isinteresting
0                0
1                0
2                1
3                1
4                0
..             ...
195              0
196              0
197              0
198              1
199              1

[200 rows x 1 columns]


In [25]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn import datasets

In [26]:
clf = svm.SVC()
clf.fit(X_train, y_train)


SVC()

In [27]:
prediction = clf.predict(X_test).astype(int)
prediction = prediction.reshape(-1, 1)
sns.histplot(prediction)

NameError: name 'sns' is not defined

In [ ]:
pip install tabulate

In [ ]:
from sklearn.metrics import confusion_matrix

y_true = y_test.astype(int)
y_pred = prediction.astype(int)

from sklearn.metrics import ConfusionMatrixDisplay

cm = confusion_matrix(y_true, y_pred)

cm_display = ConfusionMatrixDisplay(cm).plot()

from sklearn.metrics import accuracy_score

y_score = accuracy_score(y_true, y_pred)
y_score

tn, fp, fn, tp = (cm).ravel()
from tabulate import tabulate
precision = tp/(tp+fp)
table = [['True Negative', 'False Positive', 'False Negative', 'True Positive', 'Accuracy', 'Precision'], [tn, fp, fn, tp, y_score, precision]]
print(tabulate(table))
